# Computate Smart Cloud Builder

## About the open source GPL3 license and copyright for this product

Copyright © 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


# Create your project

## Watch your project for code changes


Run the Ansible Playbook to configure and compile the your project. 
This will index all of your project's Java classes into the Apache Solr search engine. 

In [ ]:
eval $(./vars.py)
ansible-playbook -e @$VARS_PATH playbooks/computate-project.yaml \
  -e SYSTEMD_ENABLED=false \
  -e SITE_NAME=$SITE_NAME \
  -e SITE_PREFIX="$SITE_PREFIX" \
  -e SOLR_HOST_NAME="$SOLR_HOST_NAME" \
  -e SOLR_PORT="$SOLR_PORT" \
  -e SOLR_SSL="$SOLR_SSL" \
  -e SOLR_URL="$SOLR_URL_COMPUTATE" \
  -e SOLR_URL_COMPUTATE="$SOLR_URL_COMPUTATE"
echo DONE

## Index all Java Classes in search engine, and generate more code at the same time

Next we will use the `computate` project with the  `ai-telemetry` project to index all of the `ai-telemetry` Java classes into the Apache Solr search engine. 
It will also generate additional necessary Java classes along the way, because the computate project writes lots of code for you. 
Be patient during this part, it's generating hundreds of thousands of lines of code. 
It should take about 4 minutes. 

In [ ]:
./generate.sh
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `ai-telemetry` project. 

In [ ]:
eval $(./vars.py)
curl -s -u "$SOLR_USERNAME:$SOLR_PASSWORD" "$SOLR_URL_COMPUTATE/query?rows=0&fq=siteNom_indexed_string:$SITE_NAME" -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

## Run SQL create scripts with new PostgreSQL tables
When we create new Smart Data Models that are persisted in the PostgreSQL database, we need to make sure that our database has all the tables and fields required to persist the data. Run the commands below to rsync the `db-create.sql` script to the PostgreSQL pod. It will connect again and apply all of the database schema changes to the database. 

In [ ]:
eval $(./vars.py)
oc exec pod/postgresql-0 -- mkdir -p /bitnami/postgresql/sql/
oc rsync $SITE_SRC/src/main/resources/sql/ \
  pod/postgresql-0:/bitnami/postgresql/sql/
oc exec pod/postgresql-0 -- \
  env PGPASSWORD=$(oc -n $SITE_NAMESPACE get secret/postgres-pguser-$DATABASE_USERNAME -o jsonpath={.data.password} | base64 -d) \
  psql -U $DATABASE_USERNAME -d $DATABASE_DATABASE -f /bitnami/postgresql/sql/db-create.sql
echo DONE

## Compile your Java project

In [ ]:
eval $(./vars.py)
(cd $SITE_SRC && mvn clean install)
echo DONE

## Run your Java project

To run and debug the project in VSCode in your workbench, create a `.vscode` directory inside of your home directory. 

In [ ]:
mkdir $HOME/.vscode/
ls -ld $HOME/.vscode/
echo DONE

Create a `launch.json` file inside of the `.vscode directory`. 

In [ ]:
cat > $HOME/.vscode/launch.json <<EOF
{
  "version": "0.2.0",
  "configurations": [
    {
      "type": "java",
      "name": "MainVerticle",
      "request": "launch",
      "mainClass": "org.mghpcc.aitelemetry.verticle.MainVerticle",
      "projectName": "aitelemetry"
      , "env": {
        "VARS_PATH": "/opt/app-root/src/ai-telemetry/doc/getting-started/openshift-ai/vars.yaml"
      }
    }
  ]
}
EOF

To run and debug the project: 

- You will need to first install the `Debugger for Java` in the extension VSCode menu on the left. 
- Now you can run the project from the `Run and Debug` menu of VSCode on the left. 

Optionally, you can run the command below in a terminal. 

## Create kustomize resources to expose your site in development
Create a directory inside of the project to store your kustomize resources for development. 

In [ ]:
eval $(./vars.py)
mkdir $SITE_SRC/kustomize/overlays/$OPENSHIFT_AI_WORKBENCH/$SITE_NAME/
ls -ld $SITE_SRC/kustomize/overlays/$OPENSHIFT_AI_WORKBENCH/$SITE_NAME/
echo DONE

## Create a kustomization.yaml file

In [ ]:
eval $(./vars.py)
cat > $SITE_SRC/kustomize/overlays/$OPENSHIFT_AI_WORKBENCH/$SITE_NAME/kustomization.yaml <<EOF
apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization
resources:
- service.yaml
- route.yaml
- networkpolicy.yaml
EOF
echo DONE

## Expose a route to your development site

Create a route to your VSCode workbench to expose your development site. 

In [ ]:
eval $(./vars.py)
cat > $SITE_SRC/kustomize/overlays/$OPENSHIFT_AI_WORKBENCH/$SITE_NAME/route.yaml <<EOF
kind: Route
apiVersion: route.openshift.io/v1
metadata:
  name: vscode-$OPENSHIFT_AI_WORKBENCH
  labels:
    notebook-name: $OPENSHIFT_AI_WORKBENCH
spec:
  host: $OPENSHIFT_AI_WORKBENCH-ai-telemetry.apps.shift.nerc.mghpcc.org
  to:
    kind: Service
    name: vscode-$OPENSHIFT_AI_WORKBENCH
    weight: 100
  port:
    targetPort: 10210
  tls:
    termination: edge
  wildcardPolicy: None
EOF
echo DONE

## Expose a network policy to your development site

Create a network policy to your VSCode workbench to expose your development site. 

In [ ]:
eval $(./vars.py)
cat > $SITE_SRC/kustomize/overlays/$OPENSHIFT_AI_WORKBENCH/$SITE_NAME/networkpolicy.yaml <<EOF
kind: NetworkPolicy
apiVersion: networking.k8s.io/v1
metadata:
  name: vscode-$OPENSHIFT_AI_WORKBENCH
spec:
  podSelector:
    matchLabels:
      notebook-name: $OPENSHIFT_AI_WORKBENCH
  ingress:
    - ports:
        - protocol: TCP
          port: 10210
  policyTypes:
    - Ingress
EOF
echo DONE

## Expose a service to your development site

Create a service to your VSCode workbench to expose your development site. 

In [ ]:
eval $(./vars.py)
cat > $SITE_SRC/kustomize/overlays/$OPENSHIFT_AI_WORKBENCH/$SITE_NAME/service.yaml <<EOF
kind: Service
apiVersion: v1
metadata:
  name: vscode-$OPENSHIFT_AI_WORKBENCH
spec:
  ipFamilies:
    - IPv4
  ports:
    - name: dev
      protocol: TCP
      port: 10210
      targetPort: 10210
  internalTrafficPolicy: Cluster
  type: ClusterIP
  ipFamilyPolicy: SingleStack
  sessionAffinity: None
  selector:
    notebook-name: $OPENSHIFT_AI_WORKBENCH
EOF
echo DONE

## Deploy your new kustomize resources for development

In [ ]:
eval $(./vars.py)
oc apply -k $SITE_SRC/kustomize/overlays/$OPENSHIFT_AI_WORKBENCH/$SITE_NAME/
echo DONE